In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


## Import Modules

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Loading the dataset

In [ ]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
train.head()

In [ ]:
## statistical info
train.describe()

In [ ]:
## datatype info
train.info()

## Exploratory Data Analysis

In [ ]:
## categorical attributes
sns.countplot(train['Survived'])

In [ ]:
sns.countplot(train['Pclass'])

In [ ]:
sns.countplot(train['Sex'])

In [ ]:
sns.countplot(train['SibSp'])

In [ ]:
sns.countplot(train['Parch'])

In [ ]:
sns.countplot(train['Embarked'])

In [ ]:
## numerical attributes
sns.distplot(train['Age'])

In [ ]:
sns.distplot(train['Fare'])

In [ ]:
class_fare = train.pivot_table(index='Pclass', values='Fare')
class_fare.plot(kind='bar')
plt.xlabel('Pclass')
plt.ylabel('Avg. Fare')
plt.xticks(rotation=0)
plt.show()

In [ ]:
class_fare = train.pivot_table(index='Pclass', values='Fare', aggfunc=np.sum)
class_fare.plot(kind='bar')
plt.xlabel('Pclass')
plt.ylabel('Total Fare')
plt.xticks(rotation=0)
plt.show()

In [ ]:
sns.barplot(data=train, x='Pclass', y='Fare', hue='Survived')

In [ ]:
sns.barplot(data=train, x='Survived', y='Fare', hue='Pclass')

## Data Preprocessing

In [ ]:
train_len = len(train)
# combine two dataframes
df = pd.concat([train, test], axis=0)
df = df.reset_index(drop=True)
df.head()

In [ ]:
df.tail()

In [ ]:
## find the null values
df.isnull().sum()

In [ ]:
# drop or delete the column
df = df.drop(columns=['Cabin'], axis=1)

In [ ]:
df['Age'].mean()

In [ ]:
# fill missing values using mean of the numerical column
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())

In [ ]:
df['Embarked'].mode()[0]

In [ ]:
# fill missing values using mode of the categorical column
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

## Log transformation for uniform data distribution

In [ ]:
sns.distplot(df['Fare'])

In [ ]:
df['Fare'] = np.log(df['Fare']+1)

In [ ]:
sns.distplot(df['Fare'])

## Correlation Matrix

In [ ]:
corr = df.corr()
plt.figure(figsize=(15, 9))
sns.heatmap(corr, annot=True, cmap='coolwarm')

In [ ]:
df.head()

In [ ]:
## drop unnecessary columns
df = df.drop(columns=['Name', 'Ticket'], axis=1)
df.head()

## Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = ['Sex', 'Embarked']
le = LabelEncoder()

for col in cols:
    df[col] = le.fit_transform(df[col])
df.head()

## Train-Test Split

In [ ]:
train = df.iloc[:train_len, :]
test = df.iloc[train_len:, :]

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# input split
X = train.drop(columns=['PassengerId', 'Survived'], axis=1)
y = train['Survived']

In [ ]:
X.head()

## Model Training

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
# classify column
def classify(model):
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    model.fit(x_train, y_train)
    print('Accuracy:', model.score(x_test, y_test))
    
    score = cross_val_score(model, X, y, cv=5)
    print('CV Score:', np.mean(score))

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
classify(model)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
classify(model)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
classify(model)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
classify(model)

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
classify(model)

In [ ]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
classify(model)

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(verbose=0)
classify(model)

## Complete Model Training with Full Data

In [ ]:
model = LGBMClassifier()
model.fit(X, y)

In [ ]:
test.head()

In [ ]:
# input split for test data
X_test = test.drop(columns=['PassengerId', 'Survived'], axis=1)

In [ ]:
X_test.head()

In [ ]:
pred = model.predict(X_test)
pred

## Test Submission

In [ ]:
sub = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
sub.head()

In [ ]:
sub.info()

In [ ]:
sub['Survived'] = pred
sub['Survived'] = sub['Survived'].astype('int')

In [ ]:
sub.info()

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)